# Find a view a snapshot

First, show a list of all snapshots taken 2018-12-19

In [1]:
from databroker import Broker
cfg = {
    'metadatastore' : {
        'module' : 'databroker.headersource.mongo',
        'class' : 'MDS',
        'config' : {
            'host': 'localhost',
             'port': 27017,
             'database': 'metadatastore-production-v1',
             'timezone': 'US/Central'},
    },
    'assets' : {
        'module' : 'databroker.assets.mongo',
        'class' : 'Registry',
        'config': {
            'host': 'localhost', 
            'port': 27017, 
            'database': 'filestore-production-v1'}
    },
}
db = Broker.from_config(cfg)

snapshots = db(plan_name="snapshot", since="2018-12-19", until="2018-12-19 23:59")
# change the generator into a list
snapshots = list(snapshots)
len(snapshots)

8

In [2]:
# show the list
for h in db(plan_name="snapshot", since="2018-12-19", until="2018-12-19 23:59"):
    print(h.start["iso8601"], h.start["uid"])

2018-12-19 15:38:34.938716 80d8ccc9-2627-4302-8794-17f9e3388887
2018-12-19 15:37:30.558283 3dd5c309-bd0b-409c-8516-90f8acb01b46
2018-12-19 15:13:25.259141 86a6e904-8ec4-4e82-a866-62926312f6f9
2018-12-19 14:43:10.392916 171f9e0c-cbbb-4b9d-8dd3-1fc90b17cb98
2018-12-19 14:42:39.254010 3a42b0e7-2323-448b-b83f-de4f78618011
2018-12-19 14:41:48.126061 1ae88395-63a1-4151-a6d8-2d71f7b70507
2018-12-19 14:38:51.118818 6b03f2e3-1167-40ce-8c13-941e98772be0
2018-12-19 14:08:52.331804 8af747ec-87fe-4115-894f-38dc452a64b5


In [3]:
# be a bit more restrictive, just the snapshots created from our jupyter notebook demo
h = list(db(plan_name="snapshot", since="2018-12-19", until="2018-12-19 23:59", purpose="jupyter notebook demo"))[0]
h.table()

,time,signal_2,signal_4,signal_1,signal_3,signal_0,signal_5
seq_num,,,,,,,
1,2018-12-19 15:38:35.316101313,29.894444,0.022906,22.200001,22.205557,29.9,0.022906


In [4]:
# decidedly terse, there _is_ more information available from the descriptor document, perhaps too much
for key, doc in h.documents():
    print(key, doc)

start 
Run Start
hints           :
iso8601         : 2018-12-19 15:38:34.938716              
plan_description: archive snapshot of ophyd Signals (usually EPICS PVs)
plan_name       : snapshot                                
plan_type       : generator                               
purpose         : jupyter notebook demo                   
scan_id         : 1                                       
time            : 1545255514.9400032                      
uid             : 80d8ccc9-2627-4302-8794-17f9e3388887    
descriptor 
Event Descriptor
configuration   :
      signal_2        : 29.89444351196289                       
      signal_2        : 1545255480.061174                       
        source          : PV:rpi5bf5:0:humidity:1m                
        dtype           : number                                  
        shape           : []                                      
        precision       : 2                                       
        units           : %        

If we want a better report, then we must build a [callback](http://nsls-ii.github.io/bluesky/callbacks.html#overview-of-callbacks) that will sift the documents and show us just the information we want (PV, value, timestamp).

# custom report using a callback

In [6]:
from bluesky.callbacks.core import CallbackBase
import datetime
import pyRestTable

class SnapshotReport(CallbackBase):
    
    xref = {}    # key=PVname, value=dict(value, iso8601 timestamp)
    
    def descriptor(self, doc):
        """
        special case:  
           the data is both in the descriptor AND the event docs
           due to the way our plan created it
        """
        self.xref = {}
        for k, v in doc["configuration"].items():
            ts = v["timestamps"][k]
            dt = datetime.datetime.fromtimestamp(ts).isoformat().replace("T", " ")
            pvname = v["data_keys"][k]["source"][3:]
            value = v["data"][k]
            self.xref[pvname] = dict(value=value, timestamp=dt)
    
    def event(self, doc):
        pass
    
    def stop(self, doc):
        t = pyRestTable.Table()
        t.addLabel("EPICS PV")
        t.addLabel("value")
        t.addLabel("timestamp")
        for k, v in sorted(self.xref.items()):
            t.addRow((k, v["value"], v["timestamp"]))
        print(t)
    
    def print_report(self, header):
        for key, doc in header.documents():
            self(key, doc)        


snapshot_report = SnapshotReport()
snapshot_report.print_report(h)


============================= =================== ==========================
EPICS PV                      value               timestamp                 
============================= =================== ==========================
rpi5bf5:0:humidity            29.899999618530273  2018-12-19 15:38:32.117445
rpi5bf5:0:humidity:1m         29.89444351196289   2018-12-19 15:38:00.061174
rpi5bf5:0:humidity:1m:sdev    0.02290622889995575 2018-12-19 15:38:00.061416
rpi5bf5:0:temperature         22.200000762939453  2018-12-19 15:38:32.117511
rpi5bf5:0:temperature:1m      22.205556869506836  2018-12-19 15:38:00.061292
rpi5bf5:0:temperature:1m:sdev 0.02290579304099083 2018-12-19 15:38:00.061549
============================= =================== ==========================



## Further analysis

For further work with data from the databroker, see this [documentation](http://nsls-ii.github.io/databroker/tutorial.html#load-data-as-a-table).